![Practicum AI Logo image](https://github.com/PracticumAI/practicumai.github.io/blob/main/images/logo/PracticumAI_logo_250x50.png?raw=true)
***
# *Practicum AI:* Keras Transfer Learning
This exercise adapted from Krohn, J. and Beyleveld, G (2020) <i>Deep Learning Illustrated: A Visual, Interactive Guide to Artificial Intelligence </i> from <a href="https://github.com/the-deep-learners/deep-learning-illustrated/blob/master/notebooks/transfer_learning_in_keras.ipynb">Pearson Education</a> (Transfer Learning, Chapter 10).

In this notebook, we cover how to load a pre-trained model (VGGNet19) and finetune it for a new task: detecting hot dogs.

<p float="left">
  <img src="images/hot_dog.png" width="250" height="250" />
</p>

#### Load dependencies

Import the supporting libraries and VGG19 model.

In [1]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg19 import VGG19

#### Load the pre-trained VGG19 model

After the success of Alexnet in the 2012 ILSVRC competition, more complex models soon appeared.  VGGNet, for example, was the runner-up in the 2014 competition.  It has the classic convolution / pooling architecture of Alex but with more layers.

- **include_top = false** : We do not want the final dense classification layers from the original model.  These layers were trained to classify the original ImageNet data.
- **weights = 'imagenet'** : Use the model parameters trained on the 14 million-sample ImageNet dataset.
- **input_shape = (224, 224, 3)** : Initialize the model with the input image size of our hot dog / not hot dog dataset.

In [ ]:
vgg19 = VGG19(include_top = False,
              weights     = 'imagenet',
              input_shape = (224,224,3),
              pooling     = None)

#### Freeze all the layers in the base VGGNet19 model

Traverse the layers in the model and set the trainable flag to False. This freezes these layers and prevents them from being updated during training.

In [3]:
# Documents...
for layer in vgg19.layers:
    layer.trainable = False

#### Add custom classification layers

Add a new *Dense* layer on top of the original VGG19 model.

In [4]:
# Instantiate the sequential model and add the VGG19 model: 
model = Sequential()
model.add(vgg19)

# Add the custom layers atop the VGG19 model: 
model.add(Flatten(name = 'flattened'))
model.add(Dropout(0.5, name = 'dropout'))
model.add(Dense(2, activation = 'softmax', name = 'predictions'))

#### Compile the model for training

Compile the model with the Adaptive Momentum (adam) optimizer and the categorical_crossentropy loss function.  This loss function is appropriate for multi-class classification tasks where there are two or more output labels.  In this case, the label is either *hot dog* or *not hot dog*. 

In [5]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

#### Prepare the data for training

Acquire the dataset from this [location](https://www.kaggle.com/dansbecker/hot-dog-not-hot-dog/home).  And then define two data generator classes, one for training and a second for validation  The train data generator augments the data by rotating images within a 30 degree range, randomly flipping the images horizontally, rescaling the data by multiplying it by 1/255, and loading the image data into arrays in the channels_last format. 

In [7]:
# Instantiate two image generator classes:
train_datagen       = ImageDataGenerator(
    rescale         = 1.0/255,
    data_format     = 'channels_last',
    rotation_range  = 30,
    horizontal_flip = True,
    fill_mode       = 'reflect')

valid_datagen   = ImageDataGenerator(
    rescale     = 1.0/255,
    data_format = 'channels_last')

In [8]:
# Define the batch size:
batch_size = 32

The `flow_from_directory` method directs each generator to load images from the respective train and test directories.  Also, we specify that the data is categorical, divided into two classes.

In [9]:
# Define the train and validation generators: 
train_generator = train_datagen.flow_from_directory(
    directory   = './data/train',
    target_size = (224, 224),
    classes     = ['hot_dog','not_hot_dog'],
    class_mode  = 'categorical',
    batch_size  = batch_size,
    shuffle     = True,
    seed        = 42)

valid_generator = valid_datagen.flow_from_directory(
    directory   = './data/test',
    target_size = (224, 224),
    classes     = ['hot_dog','not_hot_dog'],
    class_mode  = 'categorical',
    batch_size  = batch_size,
    shuffle     = True,
    seed=42)

Found 499 images belonging to 2 classes.
Found 500 images belonging to 2 classes.


#### Train the Model

Now we train the model using the *fit_generator* method.  We use *fit_generator* instead of *fit* because the data is coming from data generators rather than arrays.

In [10]:
model.fit_generator(train_generator, steps_per_epoch = 15, 
                    epochs = 16, validation_data = valid_generator, 
                    validation_steps = 15)

/scratch/local/47771489/ipykernel_23690/2169329564.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator, steps_per_epoch=15,


Epoch 1/16


2022-09-22 16:38:36.652277: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8201
2022-09-22 16:38:37.586112: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Running ptxas --version returned 32512
2022-09-22 16:38:37.693303: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: ptxas exited with non-zero error code 32512, output: 
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.


 1/15 [=>............................] - ETA: 59s - loss: 0.9510 - accuracy: 0.5000

2022-09-22 16:38:39.339485: I tensorflow/stream_executor/cuda/cuda_blas.cc:1774] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


15/15 [==============================] - 12s 543ms/step - loss: 0.9994 - accuracy: 0.5503 - val_loss: 0.7683 - val_accuracy: 0.5979
Epoch 2/16
15/15 [==============================] - 7s 453ms/step - loss: 0.5931 - accuracy: 0.7173 - val_loss: 0.4672 - val_accuracy: 0.7750
Epoch 3/16
15/15 [==============================] - 7s 455ms/step - loss: 0.4813 - accuracy: 0.7580 - val_loss: 0.5778 - val_accuracy: 0.7188
Epoch 4/16
15/15 [==============================] - 7s 481ms/step - loss: 0.3919 - accuracy: 0.8180 - val_loss: 0.4226 - val_accuracy: 0.7958
Epoch 5/16
15/15 [==============================] - 7s 458ms/step - loss: 0.4212 - accuracy: 0.7987 - val_loss: 0.6792 - val_accuracy: 0.7104
Epoch 6/16
15/15 [==============================] - 7s 455ms/step - loss: 0.4130 - accuracy: 0.8180 - val_loss: 0.5876 - val_accuracy: 0.7521
Epoch 7/16
15/15 [==============================] - 7s 453ms/step - loss: 0.3869 - accuracy: 0.8266 - val_loss: 0.4716 - val_accuracy: 0.7896
Epoch 8/16
15/15